# Generate Data
Este notebook recoge todos los parquet utilizados para generar un dataset con todas las columnas filtradas y rellenadas.

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Cargar datos

In [35]:
construido = pd.read_parquet("construido.parquet") 
educacion = pd.read_parquet("education.parquet")   
income = pd.read_parquet("income.parquet")         
poblacion = pd.read_parquet("poblacion.parquet")   
renta = pd.read_parquet("renta.parquet")           
turismo = pd.read_parquet("tourism.parquet")       
waste = pd.read_parquet("waste.parquet")           
area = pd.read_parquet("area.parquet")

## Generar columnas de íncide (País ISO, nombre país y año)

In [36]:
yea = list(waste.YEA.unique())
yea.sort()
print(len(yea))

cou_rem = ['OECD - Total', 'OECD - Europe', 'OECD Asia Oceania', 'OECD America']
cou = waste.Country.unique()
countr = [a for a in cou if a not in cou_rem]
print(len(countr))

cou_rem = ['OECD', 'OECDE', 'OECDAO', 'OECDAM']
cou = waste.COU.unique()
COUNTR = [a for a in cou if a not in cou_rem]
print(len(COUNTR))

years = yea*len(countr)
print(len(years))

countries = [item for item in countr for i in range(len(yea))]
print(len(countries))

COUNTRIES = [item for item in COUNTR for i in range(len(yea))]
print(len(COUNTRIES))

28
43
43
1204
1204
1204


In [37]:
df = pd.DataFrame({'Country':countries, 'Year':years, 'COU':COUNTRIES})
df.head()

,Country,Year,COU
0,Australia,1990,AUS
1,Australia,1991,AUS
2,Australia,1992,AUS
3,Australia,1993,AUS
4,Australia,1994,AUS


## Terreno construido
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [38]:
df["Built"] = [np.nan]*len(COUNTRIES)
construido = construido[construido.MEAS == 'PCNT']
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        df.loc[(df.COU == a) & (df.Year == b), "Built"] = \
            float(construido[(construido.COU == a) & (construido.Year <= b)].tail(1).Value)
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built
0,Australia,1990,AUS,0.12103
1,Australia,1991,AUS,0.12103
2,Australia,1992,AUS,0.12103
3,Australia,1993,AUS,0.12103
4,Australia,1994,AUS,0.12103


## Personas sin la secundaria
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [39]:
df["Below Secundary"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if educacion[(educacion.LOCATION == a) & (educacion.TIME <= b)].empty :
            c = np.nan
        else:
            c = float(educacion[(educacion.LOCATION == a) & (educacion.TIME <= b)].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Below Secundary"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary
0,Australia,1990,AUS,0.12103,44.127056
1,Australia,1991,AUS,0.12103,44.127056
2,Australia,1992,AUS,0.12103,44.127056
3,Australia,1993,AUS,0.12103,47.159046
4,Australia,1994,AUS,0.12103,49.802025


## Mediana de ingresos
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [40]:
df["Income Median"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if income[(income.LOCATION == a) & (income.Year <= b) & (income.MEASURE == "MEDIANC")].empty :
            c = np.nan
        else:
            c = float(income[(income.LOCATION == a) & (income.Year <= b) & (income.MEASURE == "MEDIANC")].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Income Median"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median
0,Australia,1990,AUS,0.12103,44.127056,17368.0
1,Australia,1991,AUS,0.12103,44.127056,17368.0
2,Australia,1992,AUS,0.12103,44.127056,17368.0
3,Australia,1993,AUS,0.12103,47.159046,17368.0
4,Australia,1994,AUS,0.12103,49.802025,17368.0


## Poblacion total
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [41]:
df["Population"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == 'TOTAL')].empty :
            c = np.nan
        else:
            c = float(poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == 'TOTAL')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Population"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0


## Personas por encima de 65 años
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.


In [42]:
df["Over 65"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == '65_OVER')].empty :
            c = np.nan
        else:
            c = float(poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == '65_OVER')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Over 65"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df["Over 65 Percentage"] = df["Over 65"] / df["Population"]
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595


## Personas por encima de 85 años
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.


In [43]:
df["Over 85"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == '85_OVER')].empty :
            c = np.nan
        else:
            c = float(poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == '85_OVER')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Over 85"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df["Over 85 Percentage"] = df["Over 85"] / df["Population"]
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,Over 85 Percentage
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122


## Personas por encima de 50 años
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [44]:
df["Over 50"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == '50_OVER')].empty :
            c = np.nan
        else:
            c = float(poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == '50_OVER')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Over 50"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df["Over 50 Percentage"] = df["Over 50"] / df["Population"]
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,Over 85 Percentage,Over 50,Over 50 Percentage
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426


## Personas por debajo de 20 años
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [45]:
df["Under 20"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == 'LESS_20')].empty :
            c = np.nan
        else:
            c = float(poblacion[(poblacion.LOCATION == a) & (poblacion.TIME <= b) & (poblacion.SEX == 'T') & (poblacion.AGE == 'LESS_20')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Under 20"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df["Under 20 Percentage"] = df["Under 20"] / df["Population"]
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,Over 85 Percentage,Over 50,Over 50 Percentage,Under 20,Under 20 Percentage
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796


## Renta per cápita
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [46]:
df["Income Per Capita"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if renta[(renta.LOCATION == a) & (renta.TIME <= b) & (renta.SUBJECT == 'T_GDPPOP') & (renta.MEASURE == 'CPC')].empty :
            c = np.nan
        else:
            c = float(renta[(renta.LOCATION == a) & (renta.TIME <= b) & (renta.SUBJECT == 'T_GDPPOP') & (renta.MEASURE == 'CPC')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Income Per Capita"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,Over 85 Percentage,Over 50,Over 50 Percentage,Under 20,Under 20 Percentage,Income Per Capita
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,17788.024389
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,18125.690575
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,19095.935298
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,20119.821206
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,21350.873236


## Porcentaje del GDP que proviene del turismo
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [47]:
df["Tourism"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if turismo[(turismo.COUNTRY == a) & (turismo.YEAR <= b) & (turismo.KEY_IND_PC == 'KEYIND_TOURGDPPC')].empty :
            c = np.nan
        else:
            c = float(turismo[(turismo.COUNTRY == a) & (turismo.YEAR <= b) & (turismo.KEY_IND_PC == 'KEYIND_TOURGDPPC')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Tourism"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,Over 85 Percentage,Over 50,Over 50 Percentage,Under 20,Under 20 Percentage,Income Per Capita,Tourism
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,17788.024389,2.907
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,18125.690575,2.907
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,19095.935298,2.907
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,20119.821206,2.907
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,21350.873236,2.907


## Basura municipal
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [48]:
df["Municipal"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'MUNICIPAL')].empty :
            c = np.nan
        else:
            c = float(waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'MUNICIPAL')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Municipal"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,Over 85 Percentage,Over 50,Over 50 Percentage,Under 20,Under 20 Percentage,Income Per Capita,Tourism,Municipal
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,17788.024389,2.907,12000.0
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,18125.690575,2.907,12000.0
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,19095.935298,2.907,12000.0
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,20119.821206,2.907,12000.0
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,21350.873236,2.907,12000.0


## Basura doméstica
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [49]:
df["Household"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'HOUSEHOLD')].empty :
            c = np.nan
        else:
            c = float(waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'HOUSEHOLD')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Household"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,Over 85 Percentage,Over 50,Over 50 Percentage,Under 20,Under 20 Percentage,Income Per Capita,Tourism,Municipal,Household
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,17788.024389,2.907,12000.0,7000.0
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,18125.690575,2.907,12000.0,7000.0
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,19095.935298,2.907,12000.0,7000.0
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,20119.821206,2.907,12000.0,7000.0
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,21350.873236,2.907,12000.0,7000.0


## Basura recuperada
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [50]:
df["Recovered"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'RECOVERY')].empty :
            c = np.nan
        else:
            c = float(waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'RECOVERY')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Recovered"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,Over 85 Percentage,Over 50,Over 50 Percentage,Under 20,Under 20 Percentage,Income Per Capita,Tourism,Municipal,Household,Recovered
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,17788.024389,2.907,12000.0,7000.0,2701.0
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,18125.690575,2.907,12000.0,7000.0,2701.0
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,19095.935298,2.907,12000.0,7000.0,2701.0
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,20119.821206,2.907,12000.0,7000.0,2701.0
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,0.015122,6101996.0,0.302426,5362923.0,0.265796,21350.873236,2.907,12000.0,7000.0,2701.0


## Basura reciclada
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [51]:
df["Recicled"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'RECYCLING')].empty :
            c = np.nan
        else:
            c = float(waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'RECYCLING')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Recicled"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,...,Over 50,Over 50 Percentage,Under 20,Under 20 Percentage,Income Per Capita,Tourism,Municipal,Household,Recovered,Recicled
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,6101996.0,0.302426,5362923.0,0.265796,17788.024389,2.907,12000.0,7000.0,2701.0,2701.0
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,6101996.0,0.302426,5362923.0,0.265796,18125.690575,2.907,12000.0,7000.0,2701.0,2701.0
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,6101996.0,0.302426,5362923.0,0.265796,19095.935298,2.907,12000.0,7000.0,2701.0,2701.0
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,6101996.0,0.302426,5362923.0,0.265796,20119.821206,2.907,12000.0,7000.0,2701.0,2701.0
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,6101996.0,0.302426,5362923.0,0.265796,21350.873236,2.907,12000.0,7000.0,2701.0,2701.0


## Basura compostada
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [52]:
df["Compost"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'COMPOST')].empty :
            c = np.nan
        else:
            c = float(waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'COMPOST')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Compost"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,...,Over 50 Percentage,Under 20,Under 20 Percentage,Income Per Capita,Tourism,Municipal,Household,Recovered,Recicled,Compost
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,0.302426,5362923.0,0.265796,17788.024389,2.907,12000.0,7000.0,2701.0,2701.0,814.0
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,0.302426,5362923.0,0.265796,18125.690575,2.907,12000.0,7000.0,2701.0,2701.0,814.0
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,0.302426,5362923.0,0.265796,19095.935298,2.907,12000.0,7000.0,2701.0,2701.0,814.0
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,0.302426,5362923.0,0.265796,20119.821206,2.907,12000.0,7000.0,2701.0,2701.0,814.0
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,0.302426,5362923.0,0.265796,21350.873236,2.907,12000.0,7000.0,2701.0,2701.0,814.0


## Basura desechada
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [53]:
df["Disposal"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'DISPOSAL')].empty :
            c = np.nan
        else:
            c = float(waste[(waste.COU == a) & (waste.YEA <= b) & (waste.VAR == 'DISPOSAL')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Disposal"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,...,Under 20,Under 20 Percentage,Income Per Capita,Tourism,Municipal,Household,Recovered,Recicled,Compost,Disposal
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,5362923.0,0.265796,17788.024389,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,5362923.0,0.265796,18125.690575,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,5362923.0,0.265796,19095.935298,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,5362923.0,0.265796,20119.821206,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,5362923.0,0.265796,21350.873236,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0


## Area
Cuidado, toma el valor anterior conocido, no interpola ni nada, en caso de no conocer ningún dato anterior, coge el siguente.

In [54]:
df["Area"] = [0]*len(COUNTRIES)
for a in list(df.COU.unique()):
    for b in list(df.Year.unique()):
        if area[(area.COU == a) & (area.YEA <= b) & (area.VAR == 'AREA')].empty :
            c = np.nan
        else:
            c = float(area[(area.COU == a) & (area.YEA <= b) & (area.VAR == 'AREA')].tail(1).Value)
        df.loc[(df.COU == a) & (df.Year == b), "Area"] = c
df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)
df["Built Area"] = df["Built"] * df["Area"]
df.head()

,Country,Year,COU,Built,Below Secundary,Income Median,Population,Over 65,Over 65 Percentage,Over 85,...,Income Per Capita,Tourism,Municipal,Household,Recovered,Recicled,Compost,Disposal,Area,Built Area
0,Australia,1990,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,17788.024389,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0,7741220.0,936919.8566
1,Australia,1991,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,18125.690575,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0,7741220.0,936919.8566
2,Australia,1992,AUS,0.12103,44.127056,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,19095.935298,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0,7741220.0,936919.8566
3,Australia,1993,AUS,0.12103,47.159046,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,20119.821206,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0,7741220.0,936919.8566
4,Australia,1994,AUS,0.12103,49.802025,17368.0,20176844.0,2614809.0,0.129595,305121.0,...,21350.873236,2.907,12000.0,7000.0,2701.0,2701.0,814.0,6202.0,7741220.0,936919.8566


In [55]:
df.to_parquet("dataset.parquet")